In [0]:
import os
from Bio.SeqIO.QualityIO import FastqGeneralIterator

In [0]:
cd /gpfs_fs/home/eckertlab/wbp

In [0]:
ls

In [0]:
reads = "/home/cfriedline/eckertlab/wbp/012compiled.fastq"
mprob = "/home/cfriedline/g/src/assembly-eval-code/src/dynamic/mprobability"
bowtie_build = "/home/cfriedline/g/src/bowtie2-2.2.4/bowtie2-build"
dynamic_mean = "/home/cfriedline/g/src/assembly-eval-code/src/dynamic/mean -t 1e-150"

In [0]:
contig_files = !find . -type f | grep 'contigs.fa$' | grep velvet

In [0]:
contig_files = [os.path.abspath(x) for x in contig_files]

In [0]:
contig_files

In [0]:
#only needs run once
for c in contig_files:
    if not '47' in c: #47 already done
        print("qsub bowtie_build.sh \"%s\" \"%s\"" %  (c, c))

In [0]:
seqs_per_file = 160000

In [0]:
def format_fastq_tuple(title, seq, qual):
    assert len(seq) == len(qual)
    return "@%s\n%s\n+\n%s\n" % (title, seq, qual)

In [0]:
def get_k_from_name(fname):
    return os.path.basename(os.path.dirname(fname)).split("_")[-1]

for c in contig_files:
    print(get_k_from_name(c))

In [0]:
reads

In [0]:
cd ..

In [0]:
h = open(reads)
at = 0
current = 0
writer_num = 0
out_file = None
outfiles = []
for t, s, q in FastqGeneralIterator(h):
    if not out_file:
        out_file = open("%s_%d" % (reads, writer_num), "w")
        outfiles.append(out_file.name)
        writer_num += 1
        
    if current == seqs_per_file:
        out_file.close()
        out_file = open("%s_%d" % (reads, writer_num), "w")
        outfiles.append(out_file.name)
        writer_num += 1
        current = 0
    
    out_file.write(format_fastq_tuple(t, s, q))
    
    at += 1
    current += 1
    
    if at % 100000 == 0:
        print("at %d/%d/%d (%d/file), writer %d" % (current, at, num_seqs, seqs_per_file, writer_num-1))
out_file.close()

In [0]:
total = 0
for o in outfiles:
    res = !grep -c '^\+$' {o}
    total += int(res[0])
total

In [0]:
total_reads =  95605528/4

In [0]:
assert total_reads == total

In [0]:
def ghodsi_cmd(f, a):
    return "cat %s | %s -a %s -p 30 -f fastq > %s-%s.lap" % (f, mprob, a, f, get_k_from_name(a))

In [0]:
cmds = []
for o in outfiles:
    for c in contig_files:
        cmds.append(ghodsi_cmd(o, c))

In [0]:
cmds[0]

In [0]:
scripts = []
for i, cmd in enumerate(cmds):
    script = """#!/bin/bash
#$ -cwd
#$ -pe smp 30
#$ -N ghodsi%d
#$ -V
#$ -j y
%s
    """ % (i, cmd)
    with open("ghodsi_%s.sh" % i, "w") as o:
        scripts.append(os.path.abspath(o.name))
        os.chmod(o.name, int('744', 8))
        o.write(script)

In [0]:
with open("submit.sh", "w") as o:
    os.chmod(o.name, int('744', 8))
    for s in scripts:
        o.write("qsub %s\n" % s)

In [0]:
lap_files = !ls *.lap | grep -v collapsed

In [0]:
def collapse_lap(files, name):
    with open(name, "w") as o:
        for f in files:
            for line in open(f):
                o.write(line)